# Import Dependencies

In [1]:
#data tools
import pandas as pd
from pymatgen.ext.matproj import MPRester
from pymatgen.core import Structure
#from pymatgen.vis import structure_chemview as viz

#simtool loading and interface
from simtool import findInstalledSimToolNotebooks,searchForSimTool
from simtool import getSimToolInputs,getSimToolOutputs,Run

#user interface utilities 
import os, stat
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

/apps/share64/debian7/anaconda/anaconda-6/lib/python3.7/site-packages/xarray/core/merge.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


### User Create ~/.mpkey.txt if it doesn't already exist

In [4]:
#key security
try:
    user = str(input('Paste MP API key: '))
    clear_output()
    if not user.isalnum():
        raise TypeError('Wrong Key')
    if user is None:
        raise TypeError('Empty')
    with open(os.path.expanduser('~/.mpkey.txt'), 'w') as keyfile:
        keyfile.write(user)
    os.chmod(os.path.expanduser('~/.mpkey.txt'), stat.S_IREAD | stat.S_IWRITE)
    del user
    print("Success")
except:
    print("Something seems wrong with your key")

Something seems wrong with your key


## User Prompted To pick their SemiConductor of Choice
### Choice (a): query MP and fliter on properties. Dataframe is updated with each selection.

In [2]:
with open(os.path.expanduser("~/.mpkey.txt"), "r+") as file:
    apikey = file.readline()
rester = MPRester(apikey)
sc_dicts = rester.query({ "crystal_system": "cubic"},
                        ["task_id","pretty_formula","formula","elements","e_above_hull", "spacegroup.number", "band_gap", "crystal_system"])
sc_df = pd.DataFrame(sc_dicts)

In [3]:
# define a function for visualizing input structures

def mpid_plot(mpid):

    # import POSCAR file

    struct = rester.get_structure_by_material_id(mpid, final = False, conventional_unit_cell=True)
    POSCAR_str = struct.to(fmt = "poscar")

    lines = POSCAR_str.split('\n')

    # get the lattice information

    lattice = lines[1]
    cell_vectors = np.array([lines[2].split() , lines[3].split() , lines[4].split()]).astype(float)

    # get the list of sites

    sites = []
    for line in lines[8:]:
        if not line:
            break
        sites.append([line.split()[0],line.split()[1],line.split()[2]])

    # convert from fractional to xyz

    sites = np.array(sites).astype(float)
    xyz = np.matmul(sites,cell_vectors).transpose()

    # get the coordinates of the box

    corners = np.array([[0,1,1,0,0,0,0,1,1,0,0,1,1,1,1,0,0],[0,0,1,1,0,0,0,0,1,1,0,0,0,1,1,1,1],[0,0,0,0,0,1,1,1,1,1,1,1,0,0,1,1,0]]).T
    cell = np.matmul(corners,cell_vectors).transpose()

    # get a color dictionary

    elements = lines[5].split()
    atoms = lines[6].split()
    hues = ['tab:blue','tab:orange','tab:green','tab:red']
    colors = []
    for i, atom in enumerate(atoms):
        colors.extend([elements[i]]*int(atoms[i]))

    zip_iterator = zip(elements,hues)
    color_dict = dict(zip_iterator)

    # creating figure

    plt.close('all')

    fig = plt.figure("input structure")
    ax = Axes3D(fig)

    plot = ax.scatter(xyz[0],xyz[1],xyz[2], color=[color_dict[i] for i in colors], s = 256)
    plot = ax.plot(cell[0],cell[1],cell[2],color='black')

    ax.set_title("POSCAR")
    ax.axis('off')
    
    # The following two lines generate custom fake lines that will be used as legend entries:
    markers = [plt.Line2D([0,0],[0,0],color=color, marker='o', linestyle='') for color in color_dict.values()]
    plt.legend(markers, color_dict.keys(), numpoints=1)

    # displaying the plot

    plt.show()



In [4]:
# if you know the mp-id: give it here to get a structure object
mpid = widgets.IntText(
        value=2133,
        description='MPID:',
        disabled=False
)

plot_button = widgets.Button(
    description='plot',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='plot'
)
        
def on_button_clicked(b):
    with output:
        try:
            selected_row = mpid_widget.get_changed_df().index[mpid_widget.get_selected_rows()][0]
            mpid_selected= sc_df.at[selected_row,'task_id']
            mpid.value = mpid_selected.split('-')[1]
        except:
            mpid_selected="mp-"+str(mpid.value)
        mpid_plot(mpid_selected)
        
        
def update_plot(args):
    mpid_selected="mp-"+str(mpid.value)
    mpid_plot(mpid_selected)
    
output = widgets.Output()
mpid.observe(update_plot,'value')  
plot_button.on_click(on_button_clicked)
display(mpid, plot_button, output)

# if you don't, search for your structure by filtering
mpid_widget = qgrid.show_grid(sc_df)

# filter by pretty_formula to look for a specific structure
display(mpid_widget)

IntText(value=2133, description='MPID:')

Button(description='plot', style=ButtonStyle(), tooltip='plot')

Output()

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [8]:
struct = rester.get_structure_by_material_id("mp-"+str(mpid.value), final = False, conventional_unit_cell=True)

In [9]:
#debugging struct
type(struct.sites[0].a)

numpy.float64

In [ ]:
# if you're just hunting for something we display the structure for you.
mv = viz.quick_view(struct)
mv.ball_and_stick()

In [12]:
# This is passed to the simtool to perform simulations
# structure objects obtained directly from remote database queries must be passed to simtool as dictionaries to preserve maximum numerical precision
# quantum espresso is very sensitive to precision especially for relaxation calculations.
struct_dict = struct.as_dict()#to(fmt="JSON")
struct_dict

'Pd4 N8\n1.0\n4.975000 0.000000 0.000000\n0.000000 4.975000 0.000000\n0.000000 0.000000 4.975000\nPd N\n4 8\ndirect\n0.000000 0.000000 0.000000 Pd\n0.000000 0.500000 0.500000 Pd\n0.500000 0.000000 0.500000 Pd\n0.500000 0.500000 0.000000 Pd\n0.750000 0.750000 0.750000 N\n0.750000 0.250000 0.750000 N\n0.750000 0.250000 0.250000 N\n0.750000 0.750000 0.250000 N\n0.250000 0.750000 0.250000 N\n0.250000 0.250000 0.250000 N\n0.250000 0.250000 0.750000 N\n0.250000 0.750000 0.750000 N\n'

In [11]:
#debug poscar conversion
from pymatgen.core import Structure
with open("./POSCAR", "w") as f:
    POSCAR = f.write(POSCAR_str)
    f.close()

struct2 = Structure.from_file("./POSCAR")

In [ ]:
struct2.sites[0].a

### Choice (b): upload your own poscar directly. No query necessary.

In [ ]:
#users can pass their own structure as a poscar. the simtool will take the file contents as a dictionary which is produced here
struct = Structure.from_file("./POSCAR")
struct_dict = struct.as_dict()
struct_dict

# Perform Structure Relaxation and SCF and Phonon computation and spectra extraction using simtool

### Find 670raman simtool notebook and confirm

In [15]:
#simToolName = "670raman"
simToolName = "relax_sim"
simToolLocation = searchForSimTool(simToolName)
for key in simToolLocation.keys():
    print(f"{key} = {simToolLocation[key]}")

notebookPath = /home/nanohub/nykiel.4/opt_prop/opt_prop_suite/simtool/relax_sim.ipynb
simToolName = relax_sim
simToolRevision = None
published = False


In [16]:
installedSimToolNotebooks = findInstalledSimToolNotebooks(simToolName,returnString=True)
print(installedSimToolNotebooks)

{}


### User Set Validated Inputs
670raman will automatically activate your rest api interface to Materials Project
if you have the dotfile ".mpkey.txt" in your home directory.

Otherwise, it will attempt to generate a realistic crystal structure from your chemical discription

In [17]:
#Enter your values with units! The simtool will make sure you know what you're talking about.
inputs = getSimToolInputs(simToolLocation)

In [18]:
inputs

loglevel:
    options: ['DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL']
    type: Choice
    description: python logging module level setting
    value: WARNING

numnodes:
    min: 1
    max: 8
    type: Number
    discription: Number of processors for mpi execution
    value: 1

walltime:
    type: Text
    description: Maximum time to wait for reduced job load on cluster in HH:MM:SS time format
    value: 01:00:00

pps:
    type: List
    description: List of pseudopotentials in UPF format to be used by simulator. Potentials must either exist in ./simtool/pseudo or be accessible at http://www.quantum-espresso.org/wp-content/uploads/upf_files/
    value: ['O.pbe-hgh.UPF', 'Zn.pbe-d-hgh.UPF']

smearing:
    options: ['smearing', 'fixed']
    type: Choice
    discription: Setting the extent to which there is metal-like sharing of electrons
    value: fixed

ecutwfc:
    units: rydberg
    min: 50
    max: 400
    type: Number
    description: kinetic energy cutoff for wavefunctions
   

In [19]:
# skip this: reimplementing widget definitions wip
class ToolSet():
    def __init__(self, inputs, struct_dict): #users_pp_file_list
        self.pp_list = []
        for filename in os.listdir("./simtool/pseudo/"):
            f = os.path.join("./simtool/pseudo/", filename)
            # get a list of all the PPs -- is this best instatiated here or globally?
            # if instanced here, the user could probably pass their own PPs to the constructor as well
            if os.path.isfile(f):
                self.pp_list.append(filename)

        # TODO: filter by selected compound compositions
        self.filtered_pp_list = self.pp_list

    def _create_widgets(self):
        self.log = widgets.Select(
            options=["DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"],
            value='DEBUG',
            # rows=10,
            description='Log Level:',
            disabled=False
        )
        self.walltime = widgets.Text(
            value='01:00:00',
            placeholder='walltime',
            description='walltime:',
            disabled=False
        )
        self.numnodes = widgets.IntText(
            value=8,
            placeholder='nodes',
            description='nodes:',
            disabled=False
        ) 
        self.button = widgets.Button(
            description='run simtool',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='run to submit qe simtool'
        )
        self.button.on_click(self._on_button_clicked)
        self.ecutwfc = widgets.BoundedFloatText(
            value=50,
            min=50,
            max=400,
            step=10,
            description='ecutwfc:',
            disabled=False
        )
        self.ecutrho = widgets.BoundedFloatText(
            value=200,
            min=200,
            max=1600,
            step=40,
            description='ecutrho:',
            disabled=False
        )
        self.smearing = widgets.Select(
            options=['smearing','fixed'],
            value='fixed',
            rows = 2,
            description='smearing:',
            disabled=False
        )
        self.pp_menu1 = widgets.Combobox(
            placeholder="choose a pseudopotential",
            options=filtered_pp_list,
            description='pseudopotential 1:',
            disabled=False
        )
        self.pp_menu2 = widgets.Combobox(
            placeholder="choose a pseudopotential",
            options=filtered_pp_list,
            description='pseudopotential 2:',
            disabled=False
        )
        self.output = widgets.Output()
        def _on_button_clicked(self, change):
            with output:
                print("submitting sim2l run with formula" , self.compound.value, self.spacegroup.value)
                runSim2l()
                r = Run(simToolLocation,inputs)

        # display(c, s, button, output)
        def runSim2l():
            inputs['loglevel'].value = log.value
            inputs['walltime'].value = walltime.value
            inputs['numnodes'].value = numnodes.value
            inputs['ecutwfc'].value = ecutwfc.value
            inputs['ecutrho'].value = ecutrho.value
            inputs['pps'].value = [pp_menu1.value, pp_menu2.value]
            inputs['smearing'].value = smearing.value
            inputs['struct_dict'].value = struct_dict

In [ ]:
log = widgets.Select(
    options=["DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"],
    value='DEBUG',
    # rows=10,
    description='Log Level:',

compound = widgets.Text(
    value='ZnO',
    placeholder='chemical formula',
    description='Compound:',
    disabled=False
)

walltime = widgets.Text(
    value='01:00:00',
    placeholder='walltime',
    description='walltime:',
    disabled=False
)
numnodes = widgets.IntText(
    value=8,
    placeholder='nodes',
    description='nodes:',
    disabled=False
) 
button = widgets.Button(
    description='run simtool',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='run to submit qe simtool'
)
ecutwfc = widgets.BoundedFloatText(
    value=50,
    min=50,
    max=400,
    step=10,
    description='ecutwfc:',
    disabled=False
)
ecutrho = widgets.BoundedFloatText(
    value=200,
    min=200,
    max=1600,
    step=40,
    description='ecutrho:',
    disabled=False
)
smearing = widgets.Select(
    options=['smearing','fixed'],
    value='fixed',
    rows = 2,
    description='smearing:',
    disabled=False
)
pp_menu1 = widgets.Combobox(
    placeholder="choose a pseudopotential",
    options=filtered_pp_list,
    description='pseudopotential 1:',
    disabled=False
)
pp_menu2 = widgets.Combobox(
    placeholder="choose a pseudopotential",
    options=filtered_pp_list,
    description='pseudopotential 2:',
    disabled=False
)

output = widgets.Output()

def runSim2l():
    inputs['loglevel'].value = log.value
    inputs['walltime'].value = walltime.value
    inputs['numnodes'].value = numnodes.value
    inputs['ecutwfc'].value = ecutwfc.value
    inputs['ecutrho'].value = ecutrho.value
    inputs['pps'].value = [pp_menu1.value, pp_menu2.value]
    inputs['smearing'].value = smearing.value
    inputs['struct_dict'].value = struct_dict

def on_button_clicked(b):
    with output:
        print("submitting sim2l run with formula" , compound.value)
        runSim2l()
        r = Run(simToolLocation,inputs)
        r.getResultSummary()
        print(r.read('spectra'))
        
        
button.on_click(on_button_clicked)

structure = widgets.VBox([compound,spacegroup])
simulation = widgets.VBox([ecutrho, ecutwfc, smearing, pp_menu1, pp_menu2])
run_details = widgets.VBox([walltime, numnodes, log])

accordion = widgets.VBox([widgets.Accordion(children=[simulation,run_details]),button,output])
display(accordion)

### Show User Predetermined Outputs and their Explainations

In [ ]:
outputs = getSimToolOutputs(simToolLocation)

In [ ]:
outputs

### Run simtool to obtain Predicted Raman Tensor and Spectrum Graph

In [ ]:
r.getResultSummary()

In [ ]:
print(r.read('logreport'))

In [ ]:
r.read('spectra')

In [ ]:
#check inputs
r.input_dict

In [ ]:
#find output location
print(r.outdir)